### Decimation technique

In [2]:
import numpy as np

In [3]:
lattice_constant = 3.0
interaction_range = 1
k_x = 900
k_y = 900
k_xy = 180
k_c = 900
k_c_xy = 180

In [4]:
def ranged_force_constant(lattice_constant, interaction_range, k_x, k_y, k_xy, k_c, k_c_xy, interact_potential="reciproke_squared"):
    """
    Calculate ranged force constants for the 2D Ribbon electrode dependend on which potential is used and on how many neighbors are coupled.
    
    Retruns:
        range_force_constant (list of tuples): Ranged force constant for the 2D lattice
    """

    match interact_potential:

        case "reciproke_squared":
            all_k_x = list(enumerate((k_x * (1 / (i * lattice_constant)**2) for i in range(1, interaction_range + 1))))
            all_k_y = list(enumerate((k_y * (1 / (i * lattice_constant)**2) for i in range(1, interaction_range + 1))))
            all_k_xy =  list(enumerate((k_xy * (1 / (i * lattice_constant)**2) for i in range(1, interaction_range + 1))))
            all_k_c_x = list(enumerate((k_c * (1 / (i * lattice_constant)**2) for i in range(1, interaction_range + 1))))
            all_k_c_xy = list(enumerate((k_c_xy * (1 / (i * lattice_constant)**2) for i in range(1, interaction_range + 1))))
        
        case _:
            raise ValueError("Invalid interaction potential. Choose either 'reciproke_squared', .")
        
    return all_k_x, all_k_y, all_k_xy, all_k_c_x, all_k_c_xy

In [5]:
def build_H_01_new(N_y, interaction_range, ranged_force_constants):
    """
    Build the hessian matrix for the interaction between two princial layers. The interaction range is taken into account.
    """

    N_y = N_y
    interaction_range = interaction_range

    all_k_x, all_k_y, all_k_xy = ranged_force_constants[0:3]
    h01 = np.zeros((2 * N_y * interaction_range, 2 * N_y * interaction_range))
    # build Hessian matrix for the h01 interaction between principal layers
    # rows are A layer atoms, columns are B layer atoms
    
    for i in range(h01.shape[0]):
        for j in range(h01.shape[1]):
            if i % 2 == 0 and j % 2 == 0:
                atomnr_lay1 = np.ceil(float(i + 1) / 2)
                atomnr_lay2 = np.ceil(float(j + 1) / 2)
                
                if atomnr_lay1 == atomnr_lay2:
                    h01[i, j] = -all_k_x[-1][1]
                
                if interaction_range > 1:
                    # for more than next nearest neighbour coupling in x-direction
                    for r in range(interaction_range - 1, -1, -1):
                        
                        if atomnr_lay2 == atomnr_lay1 - r * N_y:
                            h01[i, j] = -all_k_x[-(r + 1)][1]
                    
                    # xy coupling
                    if (interaction_range - 1) * N_y < atomnr_lay1 <= interaction_range * N_y and atomnr_lay2 <= N_y:
                        # edge atoms
                        if atomnr_lay1 == (interaction_range - 1) * N_y + 1 and atomnr_lay2 == 2:
                            h01[i, j] = -all_k_xy[0][1]
                            h01[i + 1, j + 1] = -all_k_xy[0][1]
                        
                        elif atomnr_lay1 == interaction_range * N_y and atomnr_lay2 == N_y - 1:
                            h01[i, j] = -all_k_xy[0][1]
                            h01[i + 1, j + 1] = -all_k_xy[0][1]
                        
                        # middle atoms
                        elif atomnr_lay2 == atomnr_lay1 - (interaction_range - 1) * N_y + 1 or atomnr_lay2 == atomnr_lay1 - (interaction_range - 1) * N_y - 1:
                            h01[i, j] = -all_k_xy[0][1]
                            h01[i + 1, j + 1] = -all_k_xy[0][1]
                    
                else:
                    # edge atoms
                    if atomnr_lay1 == 1 and atomnr_lay2 == 2:
                        h01[i, j] = -all_k_xy[0][1]
                        h01[i + 1, j + 1] = -all_k_xy[0][1]
                    elif atomnr_lay1 == N_y and atomnr_lay2 == N_y - 1:
                        h01[i, j] = -all_k_xy[0][1]
                        h01[i + 1, j + 1] = -all_k_xy[0][1]
                    elif atomnr_lay2 == atomnr_lay1 - 1 or atomnr_lay2 == atomnr_lay1 + 1:
                        h01[i, j] = -all_k_xy[0][1]
                        h01[i + 1, j + 1] = -all_k_xy[0][1]
                    
                    # make matrix now symmetric
                    h01[j, i] = h01[i, j]        
                                                

    return h01

In [6]:
interaction_range = 2
ranged_force_constants = ranged_force_constant(k_x=k_x, k_y=k_y, k_xy=k_xy, k_c=k_c, k_c_xy=k_c_xy, lattice_constant=lattice_constant, interaction_range=interaction_range)
H01 = build_H_01_new(N_y=3, interaction_range=interaction_range, ranged_force_constants=ranged_force_constants)
print(H01)

[[ -25.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.  -25.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.  -25.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [-100.    0.  -20.    0.    0.    0.  -25.    0.    0.    0.    0.    0.]
 [   0.    0.    0.  -20.    0.    0.    0.    0.    0.    0.    0.    0.]
 [ -20.    0. -100.    0.  -20.    0.    0.    0.  -25.    0.    0.    0.]
 [   0.  -20.    0.    0.    0.  -20.    0.    0.    0.    0.    0.    0.]
 [   0.    0.  -20.    0. -100.    0.    0.    0.    0.    0.  -25.    0.]
 [   0.    0.    0.  -20.    0.    0.    0.    0.    0.    0.    0.    0.]]


In [13]:
def build_H_00_new(N_y, interaction_range, ranged_force_constants):
    """
    Build the hessian matrix for the first layer. The interaction range is taken into account.

    Returns:
        H_00 (np.ndarray): Hessian matrix of shape (2 * N_y, 2 * N_y)
    """

    N_y = N_y
    interaction_range = interaction_range

    all_k_x, all_k_y, all_k_xy = ranged_force_constants[0:3]

    h00 = np.zeros((2 * N_y * interaction_range, 2 * N_y * interaction_range))

    #build Hessian matrix for the h00 principal surface layer

    for i in range(interaction_range):

        for j in range(i * 2 * N_y, i * 2 * N_y + 2 * N_y):
            
            # diagonal elements x and xy coupling
            if j % 2 == 0:

                atomnr = np.ceil(float(j + 1) / 2)
                
                # ii-coupling
                if atomnr <= N_y and interaction_range > 1:
                    ## first layer
                    h00[j, j] = sum(all_k_x[k][1] for k in range(len(all_k_x)))

                elif atomnr > i * N_y and (i + 1) * N_y == N_y * interaction_range:
                    ## last layer
                    for k in range(interaction_range):
                        if i - k > 0:
                            h00[j, j] += 2 * all_k_x[k][1]
                        else:
                            h00[j, j] += all_k_x[k][1]
                
                elif atomnr > i * N_y and atomnr <= (i + 1) * N_y and (i + 1) * N_y < N_y * interaction_range:
                    for k in range(interaction_range):
                        if i - k > 0:
                            h00[j, j] += 2 * all_k_x[k][1]
                        else:
                            h00[j, j] += all_k_x[k][1]
                            
                for k in range(interaction_range):
                    if j + 2 * (k + 1) * N_y < h00.shape[0]:                    
                        h00[j, j + 2 * (k + 1) * N_y] = -all_k_x[k][1]
                    if j - 2 * (k + 1) * N_y >= 0:
                        h00[j, j - 2 * (k + 1) * N_y] = -all_k_x[k][1]

                # xy-coupling # TODO: do something to take account that only for Ny > 1 possible or leave it to the user?
                if N_y > 1:
                    
                    if j == 0 or j == 2 * N_y - 2:
                        h00[j, j] += all_k_xy[0][1]
                        h00[j + 1, j + 1] += all_k_xy[0][1]

                    elif j < 2 * N_y - 2:
                        h00[j, j] += 2 * all_k_xy[0][1]
                        h00[j + 1, j + 1] += 2 * all_k_xy[0][1]

                    elif (j == i * 2 * N_y or j == i * 2 * N_y + 2 * N_y - 2) and (j != 0 and j != 2 * N_y - 2):
                        h00[j, j] += 2 * all_k_xy[0][1]
                        h00[j + 1, j + 1] += 2 * all_k_xy[0][1]

                    elif j != 0 + i * 2 * N_y and j != i * 2 * N_y + 2 * N_y - 2 and N_y > 2:
                        h00[j, j] = 4 * all_k_xy[0][1]
                        h00[j + 1, j + 1] += 4 * all_k_xy[0][1]
                

            else:
                
                if N_y > 1:
                    # y coupling in the coupling range -> edge layers/atoms
                    if (j == i * 2 * N_y + 1) or (j == i * 2 * N_y + 2 * N_y - 1): 
                        
                        # xy-coupling
                        atomnr = np.ceil(float(j) / 2)

                        if interaction_range > 1:
                            
                            if atomnr < N_y: #and interaction_range > 1:
                                h00[j - 1, int(j - 1 + 2 * (atomnr + N_y + 1) - 2)] = -all_k_xy[0][1]
                                h00[j, int(j - 1 + 2 * (atomnr + N_y + 1) - 1)] = -all_k_xy[0][1]

                            elif atomnr == N_y and interaction_range > 1:
                                h00[j - 1, int(2 * (atomnr + N_y - 1) - 2)] = -all_k_xy[0][1]
                                h00[j, int(2 * (atomnr + N_y - 1) - 1)] = -all_k_xy[0][1]

                            elif atomnr > N_y and interaction_range > 1 and (atomnr == (i + 1) * N_y or atomnr == i * N_y + 1) and atomnr <= N_y * interaction_range - N_y:
                                # N_y == 2 case
                                if atomnr == i * N_y + 1:
                                    h00[j, 2 * int(i * N_y + 1 + N_y + 1) - 1] = -all_k_xy[0][1]
                                    h00[j - 1, 2 * int(i * N_y + 1 + N_y + 1) - 2] = -all_k_xy[0][1]
                                    
                                    h00[j, 2 * int(i * N_y + 1 - N_y + 1) - 1] = -all_k_xy[0][1]
                                    h00[j - 1, 2 * int(i * N_y + 1 - N_y + 1) - 2] = -all_k_xy[0][1]
                                
                                elif atomnr == (i + 1) * N_y:
                                    h00[j, 2 * int((i + 1) * N_y + N_y - 1) - 1] = -all_k_xy[0][1]
                                    h00[j - 1, 2 * int((i + 1) * N_y + N_y - 1) - 2] = -all_k_xy[0][1]
                                    
                                    h00[j, 2 * int((i + 1) * N_y - N_y - 1) - 1] = -all_k_xy[0][1]
                                    h00[j - 1, 2 * int((i + 1) * N_y - N_y - 1) - 2] = -all_k_xy[0][1]
                                    
                            elif (atomnr == N_y * interaction_range - N_y + 1 or atomnr == N_y * interaction_range):
                                
                                if atomnr == N_y * interaction_range - N_y + 1:
                                    h00[j, 2 * int(N_y * interaction_range - N_y + 1 - N_y + 1) - 1] = -all_k_xy[0][1]
                                    h00[j - 1, 2 * int(N_y * interaction_range - N_y + 1 - N_y + 1) - 2] = -all_k_xy[0][1]
                                elif atomnr == N_y * interaction_range:
                                    h00[j, 2 * int(N_y * interaction_range - N_y - 1) - 1] = -all_k_xy[0][1]
                                    h00[j - 1, 2 * int(N_y * interaction_range - N_y - 1) - 2] = -all_k_xy[0][1]



                        #y - coupling
                        h00[j, j] = all_k_y[0][1] + all_k_xy[0][1]

                        if j == 1 + i * 2 * N_y:
                            h00[j, j + 2] = -all_k_y[0][1]
                        else:
                            h00[j, j - 2] = -all_k_y[0][1]

                        if interaction_range >= N_y:
                            for k in range(1, N_y - 1):
                                h00[j, j] += all_k_y[k][1]
                                
                                if j + 2 * (k + 1) < i * 2 * N_y + 2 * N_y:
                                    h00[j, j + 2 * (k + 1)] = -all_k_y[k][1]
                                if j - 2 * (k + 1) >= i * 2 * N_y:
                                    h00[j, j - 2 * (k + 1)] = -all_k_y[k][1]

                        else:
                            for k in range(1, interaction_range):
                                h00[j, j] += all_k_y[k][1]
                            
                                if j + 2 * (k + 1) < i * 2 * N_y + 2 * N_y:
                                    h00[j, j + 2 * (k + 1)] = -all_k_y[k][1]
                                if j - 2 * (k + 1) >= 0 + i * 2 * N_y:
                                    h00[j, j - 2 * (k + 1)] = -all_k_y[k][1]


                    else:
                        
                        atomnr = np.ceil(float(j) / 2)
                        h00[j, j] = 2 * all_k_y[0][1]
                        
                        # xy-coupling inner atom, inner layers
                        if interaction_range > 1:
                            
                            if atomnr < N_y: #and interaction_range > 1:
                                ## first layer
                                # first atom
                                h00[j - 1, int(2 * (atomnr + N_y - 1)) - 2] = -all_k_xy[0][1]
                                h00[j, int(2 * (atomnr + N_y - 1)) - 1] = -all_k_xy[0][1]

                                #second atom
                                h00[j - 1, int(2 * (atomnr + N_y + 1)) - 2] = -all_k_xy[0][1]
                                h00[j, int(2 * (atomnr + N_y + 1)) - 1] = -all_k_xy[0][1]

                            elif atomnr > i * N_y and (i + 1) * N_y == N_y * interaction_range:
                                ## last layer
                                # first atom
                                h00[j - 1, int(2 * (atomnr - N_y - 1)) - 2] = -all_k_xy[0][1]
                                h00[j, int(2 * (atomnr - N_y - 1)) - 1] = -all_k_xy[0][1]

                                # second atom
                                h00[j - 1, int(2 * (atomnr - N_y + 1)) - 2] = -all_k_xy[0][1]
                                h00[j, int(2 * (atomnr - N_y + 1)) - 1] = -all_k_xy[0][1]
                            
                            elif atomnr > i * N_y and atomnr < (i + 1) * N_y and (i + 1) * N_y < N_y * interaction_range:
                                ## layer before
                                # first atom
                                h00[j - 1, int(2 * (atomnr - N_y - 1)) - 2] = -all_k_xy[0][1]
                                h00[j, int(2 * (atomnr - N_y - 1)) - 1] = -all_k_xy[0][1]
                                # second atom
                                h00[j - 1, int(2 * (atomnr - N_y + 1)) - 2] = -all_k_xy[0][1]
                                h00[j, int(2 * (atomnr - N_y + 1)) - 1] = -all_k_xy[0][1]

                                ## layer after
                                # first atom
                                h00[j - 1, int(2 * (atomnr + N_y - 1)) - 2] = -all_k_xy[0][1]
                                h00[j, int(2 * (atomnr + N_y - 1)) - 1] = -all_k_xy[0][1]
                                # second atom
                                h00[j - 1, int(2 * (atomnr + N_y + 1)) - 2] = -all_k_xy[0][1]
                                h00[j, int(2 * (atomnr + N_y + 1)) - 1] = -all_k_xy[0][1]

                            

                        if j + 2 < i * 2 * N_y + 2 * N_y:
                            h00[j, j + 2] = -all_k_y[0][1]
                        if j - 2 >= 0 + i * 2 * N_y:
                            h00[j, j - 2] = -all_k_y[0][1]


                        if interaction_range >= N_y:
                            for k in range(1, N_y - 1):
                                if atomnr - k - 1 > i * N_y and atomnr + k < i * N_y + N_y:
                                    h00[j, j] += 2 * all_k_y[k][1]
                                    
                                elif (atomnr - k - 1 <= i * N_y and atomnr + k < i * N_y + N_y) or (atomnr - k - 1 > i * N_y and atomnr + k >= i * N_y + N_y):
                                    h00[j, j] += all_k_y[k][1]
                                
                                if j + 2 * (k + 1) < i * 2 * N_y + 2 * N_y:
                                    h00[j, j + 2 * (k + 1)] = -all_k_y[k][1]
                                if j - 2 * (k + 1) > i * 2 * N_y:
                                    h00[j, j - 2 * (k + 1)] = -all_k_y[k][1]

                        else:
                            for k in range(1, interaction_range):
                                if atomnr - k - 1 > i * N_y and atomnr + k < N_y + i * N_y:#N_y * interaction_range:
                                    h00[j, j] += 2 * all_k_y[k][1]
                                elif (atomnr - k - 1 <= i * N_y and atomnr + k < i * N_y + N_y) or (atomnr - k - 1 > i * N_y and atomnr + k >= i * N_y + N_y):
                                    h00[j, j] += all_k_y[k][1]
                                
                                if j + 2 * (k + 1) < i * 2 * N_y + 2 * N_y:
                                    h00[j, j + 2 * (k + 1)] = -all_k_y[k][1]
                                if j - 2 * (k + 1) >= 0 + i * 2 * N_y:
                                    h00[j, j - 2 * (k + 1)] = -all_k_y[k][1]

    return h00

In [14]:
interaction_range = 2
ranged_force_constants = ranged_force_constant(k_x=k_x, k_y=k_y, k_xy=k_xy, k_c=k_c, k_c_xy=k_c_xy, lattice_constant=lattice_constant, interaction_range=interaction_range)
H00 = build_H_00_new(N_y=3, interaction_range=interaction_range, ranged_force_constants=ranged_force_constants)
print(ranged_force_constants, '\n')
print(H00)

([(0, 100.0), (1, 25.0)], [(0, 100.0), (1, 25.0)], [(0, 20.0), (1, 5.0)], [(0, 100.0), (1, 25.0)], [(0, 20.0), (1, 5.0)]) 

[[ 145.    0.    0.    0.    0.    0. -100.    0.  -20.    0.    0.    0.]
 [   0.  145.    0. -100.    0.  -25.    0.    0.    0.  -20.    0.    0.]
 [   0.    0.  165.    0.    0.    0.  -20.    0. -100.    0.  -20.    0.]
 [   0. -100.    0.  200.    0. -100.    0.  -20.    0.    0.    0.  -20.]
 [   0.    0.    0.    0.  145.    0.    0.    0.  -20.    0. -100.    0.]
 [   0.  -25.    0. -100.    0.  145.    0.    0.    0.  -20.    0.    0.]
 [-100.    0.  -20.    0.    0.    0.  265.    0.    0.    0.    0.    0.]
 [   0.    0.    0.  -20.    0.    0.    0.  145.    0. -100.    0.  -25.]
 [ -20.    0. -100.    0.  -20.    0.    0.    0.   80.    0.    0.    0.]
 [   0.  -20.    0.    0.    0.  -20.    0. -100.    0.  200.    0. -100.]
 [   0.    0.  -20.    0. -100.    0.    0.    0.    0.    0.  265.    0.]
 [   0.    0.    0.  -20.    0.    0.    0.  -25.  

In [41]:
def build_H_NN_new(N_y, interaction_range, ranged_force_constants):
    """
    Build up an actual bulk layer of the electrode. The coupling includes options for x, y and xy coupling. The coupling range is defined by the parameter interaction_range.
    """

    N_y = N_y
    interaction_range = interaction_range

    all_k_x, all_k_y, all_k_xy = ranged_force_constants[0:3]

    hNN = np.zeros((2 * N_y * interaction_range, 2 * N_y * interaction_range))
    #build Hessian matrix for the hNN principal bulklayer

    for i in range(interaction_range):

        for j in range(i * 2 * N_y, i * 2 * N_y + 2 * N_y):
            
            # diagonal elements x and xy coupling
            if j % 2 == 0:
                
                atomnr = np.ceil(float(j + 1) / 2)
                
                # ii-coupling
                hNN[j, j] = sum(2 * all_k_x[k][1] for k in range(len(all_k_x))) 


                for k in range(interaction_range):
                    if j + 2 * (k + 1) * N_y < hNN.shape[0]:                    
                        hNN[j, j + 2 * (k + 1) * N_y] = -all_k_x[k][1]
                    if j - 2 * (k + 1) * N_y >= 0:
                        hNN[j, j - 2 * (k + 1) * N_y] = -all_k_x[k][1]

                # ij-coupling in h01

                # xy-coupling
                if N_y > 1:
                    
                    if j == i * 2 * N_y or j == i * 2 * N_y + 2 * N_y - 2:
                        hNN[j, j] += 2 * all_k_xy[0][1]
                        hNN[j + 1, j + 1] += 2 * all_k_xy[0][1]

                    if j != 0 + i * 2 * N_y and j != i * 2 * N_y + 2 * N_y - 2 and N_y > 2:
                        hNN[j, j] += 4 * all_k_xy[0][1]
                        hNN[j + 1, j + 1] += 4 * all_k_xy[0][1]
                

            else:
                if N_y > 1:
                    # y coupling in the coupling range -> edge layers
                    if (j == i * 2 * N_y + 1) or (j == i * 2 * N_y + 2 * N_y - 1): 
                        
                        # xy-coupling
                        atomnr = np.ceil(float(j) / 2)

                        if interaction_range > 1:
                            
                            if atomnr < N_y:# and interaction_range > 1:
                                hNN[j - 1, int(j - 1 + 2 * (atomnr + N_y + 1) - 2)] = -all_k_xy[0][1]
                                hNN[j, int(j - 1 + 2 * (atomnr + N_y + 1) - 1)] = -all_k_xy[0][1]

                            elif atomnr ==  N_y and interaction_range > 1:
                                hNN[j - 1, int(2 * (atomnr + N_y - 1) - 2)] = -all_k_xy[0][1]
                                hNN[j, int(2 * (atomnr + N_y - 1) - 1)] = -all_k_xy[0][1]

                            
                            elif atomnr > N_y and interaction_range > 1 and (atomnr == (i + 1) * N_y or atomnr == i * N_y + 1) and atomnr <= N_y * interaction_range - N_y:
                                # N_y == 2 case
                                if atomnr == i * N_y + 1:
                                    hNN[j, 2 * int(i * N_y + 1 + N_y + 1) - 1] = -all_k_xy[0][1]
                                    hNN[j - 1, 2 * int(i * N_y + 1 + N_y + 1) - 2] = -all_k_xy[0][1]
                                    
                                    hNN[j, 2 * int(i * N_y + 1 - N_y + 1) - 1] = -all_k_xy[0][1]
                                    hNN[j - 1, 2 * int(i * N_y + 1 - N_y + 1) - 2] = -all_k_xy[0][1]
                                
                                elif atomnr == (i + 1) * N_y:
                                    hNN[j, 2 * int((i + 1) * N_y + N_y - 1) - 1] = -all_k_xy[0][1]
                                    hNN[j - 1, 2 * int((i + 1) * N_y + N_y - 1) - 2] = -all_k_xy[0][1]
                                    
                                    hNN[j, 2 * int((i + 1) * N_y - N_y - 1) - 1] = -all_k_xy[0][1]
                                    hNN[j - 1, 2 * int((i + 1) * N_y - N_y - 1) - 2] = -all_k_xy[0][1]
                                    
                            elif (atomnr == N_y * interaction_range - N_y + 1 or atomnr == N_y * interaction_range):
                                if atomnr == N_y * interaction_range - N_y + 1:
                                    hNN[j, 2 * int(N_y * interaction_range - N_y + 1 - N_y + 1) - 1] = -all_k_xy[0][1]
                                    hNN[j - 1, 2 * int(N_y * interaction_range - N_y + 1 - N_y + 1) - 2] = -all_k_xy[0][1]
                                elif atomnr == N_y * interaction_range:
                                    hNN[j, 2 * int(N_y * interaction_range - N_y - 1) - 1] = -all_k_xy[0][1]
                                    hNN[j - 1, 2 * int(N_y * interaction_range - N_y - 1) - 2] = -all_k_xy[0][1]


                        hNN[j, j] = all_k_y[0][1] + 2 * all_k_xy[0][1]

                        if j == 1 + i * 2 * N_y:
                            hNN[j, j + 2] = -all_k_y[0][1]
                        else:
                            hNN[j, j - 2] = -all_k_y[0][1]

                        if interaction_range >= N_y:
                            for k in range(1, N_y - 1):
                                hNN[j, j] += all_k_y[k][1]
                                
                                if j + 2 * (k + 1) < i * 2 * N_y + 2 * N_y:
                                    hNN[j, j + 2 * (k + 1)] = -all_k_y[k][1]
                                if j - 2 * (k + 1) >= i * 2 * N_y:
                                    hNN[j, j - 2 * (k + 1)] = -all_k_y[k][1]

                        else:
                            for k in range(1, interaction_range):
                                hNN[j, j] += all_k_y[k][1]
                            
                                if j + 2 * (k + 1) < i * 2 * N_y + 2 * N_y:
                                    hNN[j, j + 2 * (k + 1)] = -all_k_y[k][1]
                                if j - 2 * (k + 1) >= i * 2 * N_y:
                                    hNN[j, j - 2 * (k + 1)] = -all_k_y[k][1]


                    else:
                        
                        atomnr = np.ceil(float(j) / 2)
                        hNN[j, j] = 2 * all_k_y[0][1] * 4 * k_c_xy[0][1]
                        
                        # xy-coupling inner atom
                        if interaction_range > 1:
                            if atomnr < N_y:# and interaction_range > 1:
                                ## first layer
                                # first atom
                                hNN[j - 1, int(2 * (atomnr + N_y - 1)) - 2] = -all_k_xy[0][1]
                                hNN[j, int(2 * (atomnr + N_y - 1)) - 1] = -all_k_xy[0][1]

                                #second atom
                                hNN[j - 1, int(2 * (atomnr + N_y + 1)) - 2] = -all_k_xy[0][1]
                                hNN[j, int(2 * (atomnr + N_y + 1)) - 1] = -all_k_xy[0][1]

                            elif atomnr > i * N_y and (i + 1) * N_y == N_y * interaction_range:
                                ## last layer
                                # first atom
                                hNN[j - 1, int(2 * (atomnr - N_y - 1)) - 2] = -all_k_xy[0][1]
                                hNN[j, int(2 * (atomnr - N_y - 1)) - 1] = -all_k_xy[0][1]

                                # second atom
                                hNN[j - 1, int(2 * (atomnr - N_y + 1)) - 2] = -all_k_xy[0][1]
                                hNN[j, int(2 * (atomnr - N_y + 1)) - 1] = -all_k_xy[0][1]
                            
                            elif atomnr > i * N_y and atomnr < (i + 1) * N_y and (i + 1) * N_y < N_y * interaction_range:
                                ## layer before
                                # first atom
                                hNN[j - 1, int(2 * (atomnr - N_y - 1)) - 2] = -all_k_xy[0][1]
                                hNN[j, int(2 * (atomnr - N_y - 1)) - 1] = -all_k_xy[0][1]
                                # second atom
                                hNN[j - 1, int(2 * (atomnr - N_y + 1)) - 2] = -all_k_xy[0][1]
                                hNN[j, int(2 * (atomnr - N_y + 1)) - 1] = -all_k_xy[0][1]

                                ## layer after
                                # first atom
                                hNN[j - 1, int(2 * (atomnr + N_y - 1)) - 2] = -all_k_xy[0][1]
                                hNN[j, int(2 * (atomnr + N_y - 1)) - 1] = -all_k_xy[0][1]
                                # second atom
                                hNN[j - 1, int(2 * (atomnr + N_y + 1)) - 2] = -all_k_xy[0][1]
                                hNN[j, int(2 * (atomnr + N_y + 1)) - 1] = -all_k_xy[0][1]

                            

                        if j + 2 < i * 2 * N_y + 2 * N_y:
                            hNN[j, j + 2] = -all_k_y[0][1]
                        if j - 2 >= 0 + i * 2 * N_y:
                            hNN[j, j - 2] = -all_k_y[0][1]


                        if interaction_range >= N_y:
                            for k in range(1, N_y - 1):
                                if atomnr - k - 1 > i * N_y and atomnr + k < i * N_y + N_y:
                                    hNN[j, j] += 2 * all_k_y[k][1]
                                    
                                elif (atomnr - k - 1 <= i * N_y and atomnr + k < i * N_y + N_y) or (atomnr - k - 1 > i * N_y and atomnr + k >= i * N_y + N_y):
                                    hNN[j, j] += all_k_y[k][1]
                                
                                if j + 2 * (k + 1) < i * 2 * N_y + 2 * N_y:
                                    hNN[j, j + 2 * (k + 1)] = -all_k_y[k][1]
                                if j - 2 * (k + 1) > i * 2 * N_y:
                                    hNN[j, j - 2 * (k + 1)] = -all_k_y[k][1]

                        else:
                            for k in range(1, interaction_range):
                                if atomnr - k - 1 > i * N_y and atomnr + k < N_y + i * N_y:
                                    hNN[j, j] += 2 * all_k_y[k][1]
                                elif (atomnr - k - 1 <= i * N_y and atomnr + k < i * N_y + N_y) or (atomnr - k - 1 > i * N_y and atomnr + k >= i * N_y + N_y):
                                    hNN[j, j] += all_k_y[k][1]
                                
                                if j + 2 * (k + 1) < i * 2 * N_y + 2 * N_y:
                                    hNN[j, j + 2 * (k + 1)] = -all_k_y[k][1]
                                if j - 2 * (k + 1) >= 0 + i * 2 * N_y:
                                    hNN[j, j - 2 * (k + 1)] = -all_k_y[k][1]

    return hNN

In [53]:
interaction_range = 2
ranged_force_constants = ranged_force_constant(k_x=k_x, k_y=k_y, k_xy=k_xy, k_c=k_c, k_c_xy=k_c_xy, lattice_constant=lattice_constant, interaction_range=interaction_range)
HNN = build_H_NN_new(N_y=2, interaction_range=interaction_range, ranged_force_constants=ranged_force_constants)
print(ranged_force_constants, '\n')
print(HNN)

([(0, 100.0), (1, 25.0)], [(0, 100.0), (1, 25.0)], [(0, 20.0), (1, 5.0)], [(0, 100.0), (1, 25.0)], [(0, 20.0), (1, 5.0)]) 

[[ 290.    0.    0.    0. -100.    0.  -20.    0.]
 [   0.  140.    0. -100.    0.    0.    0.  -20.]
 [   0.    0.  290.    0.  -20.    0. -100.    0.]
 [   0. -100.    0.  140.    0.  -20.    0.    0.]
 [-100.    0.  -20.    0.  290.    0.    0.    0.]
 [   0.    0.    0.  -20.    0.  140.    0. -100.]
 [ -20.    0. -100.    0.    0.    0.  290.    0.]
 [   0.  -20.    0.    0.    0. -100.    0.  140.]]


# Test Grid + Vectors

In [38]:
def construct_hessian_matrix(nx, ny, couplings):
    """
    Konstruiert die Hesse-Matrix für ein 2D-Gitter.

    Args:
        nx (int): Anzahl der Atome in x-Richtung.
        ny (int): Anzahl der Atome in y-Richtung.
        couplings (list of tuple): Eine Liste von Kopplungen. Jedes Element ist
                                   ein Tupel (k, dx, dy, interaction_type), wobei:
                                   - k (float): Die Kopplungskonstante.
                                   - dx (int): Der Abstandsvektor in x-Richtung.
                                   - dy (int): Der Abstandsvektor in y-Richtung.
                                   - interaction_type (str): 'x', 'y' oder 'xy'
                                     für die Richtung der Feder.
    
    Returns:
        numpy.ndarray: Die Hesse-Matrix.
    """
    num_atoms = nx * ny
    dim = 2
    matrix_size = num_atoms * dim
    
    # Initialisiere die Hesse-Matrix mit Nullen
    H = np.zeros((matrix_size, matrix_size))
    
    # Hilfsfunktion zur Umrechnung von Gitterkoordinaten in Matrix-Indizes
    def get_indices(i, j):
        atom_idx = i * ny + j
        x_idx = atom_idx * dim
        y_idx = atom_idx * dim + 1
        return x_idx, y_idx

    # Iteriere über alle Atome im Gitter
    for i in range(nx):
        for j in range(ny):
            atom_idx_i_x, atom_idx_i_y = get_indices(i, j)

            # Iteriere über alle Kopplungen
            for k, dx, dy, interaction_type in couplings:
                # Berechne die Koordinaten des Nachbaratoms
                neighbor_i = i + dx
                neighbor_j = j + dy
                
                # Überprüfe, ob der Nachbar innerhalb des Gitters liegt
                if 0 <= neighbor_i < nx and 0 <= neighbor_j < ny:
                    atom_idx_j_x, atom_idx_j_y = get_indices(neighbor_i, neighbor_j)

                    # Update die Diagonalelemente (i,i)
                    if interaction_type == 'x':
                        H[atom_idx_i_x, atom_idx_i_x] += k
                        H[atom_idx_j_x, atom_idx_j_x] += k
                        
                        # Update die Nicht-Diagonalelemente (i,j) und (j,i)
                        H[atom_idx_i_x, atom_idx_j_x] -= k
                        H[atom_idx_j_x, atom_idx_i_x] -= k

                    elif interaction_type == 'y':
                        H[atom_idx_i_y, atom_idx_i_y] += k
                        H[atom_idx_j_y, atom_idx_j_y] += k

                        # Update die Nicht-Diagonalelemente (i,j) und (j,i)
                        H[atom_idx_i_y, atom_idx_j_y] -= k
                        H[atom_idx_j_y, atom_idx_i_y] -= k
                    
                    elif interaction_type == 'xy':
                        H[atom_idx_i_x, atom_idx_i_x] += k
                        H[atom_idx_i_y, atom_idx_i_y] += k
                        H[atom_idx_j_x, atom_idx_j_x] += k
                        H[atom_idx_j_y, atom_idx_j_y] += k
                        
                        # Update die Nicht-Diagonalelemente (i,j) und (j,i)
                        H[atom_idx_i_x, atom_idx_j_x] -= k
                        H[atom_idx_j_x, atom_idx_i_x] -= k
                        H[atom_idx_i_y, atom_idx_j_y] -= k
                        H[atom_idx_j_y, atom_idx_i_y] -= k
    
    # Rückgabe der symmetrisierten Matrix (für Rundungsfehler)
    return (H + H.T) / 2

In [ ]:
# Gittergröße
nx = 2
ny = 2

# Kopplungskonstanten
k_x = 100.0  # Horizontale Kopplung
k_y = 100.0  # Vertikale Kopplung
k_xy = 20.0 # Diagonale Kopplung

# Definition der Kopplungen (Nachbarvektoren)
couplings = [
    # Nächste Nachbarn
    (k_x, 1, 0, 'x'),   # Horizontale Kopplung (nach rechts)
    (k_y, 0, 1, 'y'),   # Vertikale Kopplung (nach oben)
    
    # Diagonale Nachbarn
    (k_xy, 1, 1, 'xy'), # Diagonale Kopplung (oben rechts)
]

# Konstruiere die Hesse-Matrix
H_2x2 = construct_hessian_matrix(nx, ny, couplings)

print("Hesse-Matrix für ein 2x2-Gitter:")
print(H_2x2.shape)
print(np.round(H_2x2, 2))

Hesse-Matrix für ein 2x2-Gitter:
(12, 12)
[[ 120.    0.    0.    0.    0.    0. -100.    0.  -20.    0.    0.    0.]
 [   0.  120.    0. -100.    0.    0.    0.    0.    0.  -20.    0.    0.]
 [   0.    0.  120.    0.    0.    0.    0.    0. -100.    0.  -20.    0.]
 [   0. -100.    0.  220.    0. -100.    0.    0.    0.    0.    0.  -20.]
 [   0.    0.    0.    0.  100.    0.    0.    0.    0.    0. -100.    0.]
 [   0.    0.    0. -100.    0.  100.    0.    0.    0.    0.    0.    0.]
 [-100.    0.    0.    0.    0.    0.  100.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.  100.    0. -100.    0.    0.]
 [ -20.    0. -100.    0.    0.    0.    0.    0.  120.    0.    0.    0.]
 [   0.  -20.    0.    0.    0.    0.    0. -100.    0.  220.    0. -100.]
 [   0.    0.  -20.    0. -100.    0.    0.    0.    0.    0.  120.    0.]
 [   0.    0.    0.  -20.    0.    0.    0.    0.    0. -100.    0.  120.]]
